In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline


In [5]:
def symbol_to_path(symbol, base_dir=None): 
    return ("C:\\Users\\S\\github\\ML4T\\mafi_ML\\ML4T_2022Spr\\ML4T_2022Spr\\data\\{}.csv".format(str(symbol)))  
 

def get_data(symbols,dates, addSPY = True, colname = 'Adj Close'):
    df = pd.DataFrame(index = dates)
    if addSPY and "SPY" not in symbols:
        symbols =['SPY'] + list(symbols)
    for symbol in symbols:
        df_temp = pd.read_csv(symbol_to_path(symbol), index_col="Date", parse_dates=True, usecols=["Date", colname],na_values=["nan"])
        df_temp = df_temp.rename(columns={colname: symbol})
        df = df.join(df_temp)
        if symbol == "SPY": 
            df = df.dropna(subset=["SPY"])
        
    return df    

In [6]:
def getprices(symbol, start_date, end_date):
    prices = get_data(symbol, pd.date_range(start_date, end_date))
    if 'SPY' not in symbol:
            prices.drop('SPY', axis = 1, inplace = True)
         
    prices= prices.fillna(method ='ffill')
    prices = prices.fillna(method = 'bfill')
    prices_normed = prices/prices.iloc[0]
    return prices_normed 
    
#Indicator 1 - Simple Moving Average    
def getsma(prices, moving_window):
    sma = prices.rolling(window = moving_window).mean()
    sma_by_price = sma / prices
    sma_50_days = prices.rolling(window = 50).mean()
    return sma,sma_by_price,sma_50_days

#Indicator 2 - Bollinger Bands Percentage
def getBBP (prices, moving_window):
    rolling_mean = prices.rolling(window = moving_window).mean()
    rolling_std = prices.rolling(window = moving_window).std()
    upperband = rolling_mean + (2*rolling_std)
    lowerband = rolling_mean - (2*rolling_std)
    BBP = (prices - lowerband)/(upperband - lowerband)
    return upperband,lowerband, BBP

#Indicator 3 - Momentum
def getMomentum(prices, moving_window):
    momentum = prices/prices.shift(moving_window) - 1
    return momentum

#Indicator 4 - Volatility
def getVolatility(prices, moving_window):
    daily_returns = prices.copy()
    daily_returns[1:] = (prices[1:] / prices[:-1].values) - 1
    daily_returns = daily_returns[1:]
    volatility = daily_returns.rolling(window = moving_window).std()  
    return volatility

#Indicator 5  - MACD 
def getMACD(prices):
    ema_short = prices.ewm(span = 12, adjust= False).mean()
    ema_long = prices.ewm(span = 26, adjust = False).mean()
    MACD = ema_long - ema_short
    signal = MACD.ewm(span=9, adjust=False).mean() 
    return MACD, signal

In [7]:
def test_code():
    start_date = dt.datetime(2008,1,1)
    end_date = dt.datetime(2009,12,31)
    symbol ='JPM'
    moving_window = 14
    
    prices = getprices([symbol], start_date, end_date)
    
    #plotting SMA 
    sma, sma_by_price,sma_50_days = getsma(prices, moving_window)   
    fig, ax = plt.subplots(figsize=(15, 7))
    ax.set(xlabel='Time', ylabel="Price", title="Simple Moving Average")
    ax.plot(prices, "goldenrod", label="Normalized Price")
    ax.plot(sma,  "magenta", label="SMA")
    ax.legend(loc = "best")
    fig.savefig('Indicator1_SMA.png')
    plt.close()
    
    #plotting short term SMA vs Long term SMA   
    fig, ax = plt.subplots(figsize=(15, 7))
    ax.set(xlabel='Time', ylabel="Price", title="Cross Over Simple Moving Average")
    ax.plot(prices, "goldenrod", label="Normalized Price")
    ax.plot(sma, "magenta", label=f"SMA {moving_window} days")
    plt.plot(sma_50_days,'gray', label ='SMA 50 days')
    ax.legend(loc = "best")
    fig.savefig('Indicator1_crossover_SMA.png') 
    plt.close()
    
    
    #plotting Bollinger Bands
    upperband, lowerband, BBP = getBBP(prices, moving_window)   
    fig, ax = plt.subplots(figsize=(15, 7))
    ax.set(xlabel='Time', ylabel="Price", title="Bollinger Bands")
    ax.plot(prices, "goldenrod", label = 'Normalized Price')
    ax.plot(sma, "magenta", label="Rolling Average")
    ax.plot(upperband, "slateblue", label="Upper Band")
    ax.plot(lowerband, "deepskyblue",label="Lower Band")
    ax.legend()
    fig.savefig('Indicator2_BB.png')
    plt.close()
    
    #plotting Bollinger Band Percentage 
    fig, ax = plt.subplots(figsize=(15, 7))
    ax.set(xlabel='Time', ylabel="Price", title="Bollinger Bands Percentage")
    ax.plot(prices, "goldenrod",label = 'Normalized Price')
    ax.plot(BBP, "gray", label = 'BBP%')
    ax.legend()
    fig.savefig('Indicator2_BBP.png')
    plt.close()
    
    #plotting Momentum
    momentum = getMomentum(prices, moving_window)   
    fig, ax = plt.subplots(figsize=(15, 7))
    ax.set(xlabel='Time', ylabel="Price", title="Momentum")
    ax.plot(prices, "goldenrod", label = 'Normalized Price')
    ax.plot(momentum, "blue",label="Momentum")
    ax.legend()
    fig.savefig('Indicator3_Momentum.png')
    plt.close()
    
    #plotting Volatility
    volatility = getVolatility(prices,moving_window)   
    fig, ax = plt.subplots(figsize=(15, 7))
    ax.set(xlabel='Time', ylabel="Price", title="Volatility")
    ax.plot(prices, "goldenrod", label = 'Normalized Price')
    ax.plot(volatility, label="Volatility")
    ax.legend()
    fig.savefig('Indicator4_Volatility.png')
    plt.close()
    
    #plotting MACD
    MACD, signal = getMACD(prices)   
    fig, ax = plt.subplots(figsize=(15, 7))
    ax.set(xlabel='Time', ylabel="Price", title="MACD")
    ax.plot(MACD, "gray", label = 'MACD')
    ax.plot(signal, "coral", label="MACD Signal")
    ax.legend()
    fig.savefig('Indicator5_MACD.png')
    plt.close()
    

In [8]:
if __name__ == "__main__":
    test_code()

In [9]:
start_date = dt.datetime(2008,1,1)
end_date = dt.datetime(2009,12,31)
symbol ='JPM'
moving_window = 14

prices = getprices([symbol], start_date, end_date)

In [10]:
prices

,JPM
2008-01-02,1.000000
2008-01-03,0.992982
2008-01-04,0.970367
2008-01-07,0.980244
2008-01-08,0.941253
...,...
2009-12-24,1.037432
2009-12-28,1.033273
2009-12-29,1.027554
2009-12-30,1.028594


In [11]:
daily_returns = prices.copy()
daily_returns[1:] = (prices[1:] / prices[:-1].values) - 1
daily_returns = daily_returns[1:]


In [12]:
daily_returns

,JPM
2008-01-03,-0.007018
2008-01-04,-0.022775
2008-01-07,0.010179
2008-01-08,-0.039777
2008-01-09,0.014085
...,...
2009-12-24,0.007828
2009-12-28,-0.004009
2009-12-29,-0.005535
2009-12-30,0.001012


,JPM
2008-01-03,NaN
2008-01-04,NaN
2008-01-07,NaN
2008-01-08,NaN
2008-01-09,NaN
...,...
2009-12-24,0.014918
2009-12-28,0.014570
2009-12-29,0.014666
2009-12-30,0.014663
